<a href="https://colab.research.google.com/github/ykkim77/nlp-12th/blob/main/nlp_12th_week_%EC%85%B0%EC%9D%B5%EC%8A%A4%ED%94%BC%EC%96%B4%EA%B8%80%EC%83%9D%EC%84%B1_NLG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
import numpy as np
import os
from urllib.request import urlretrieve

urlretrieve('https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt','shakespeare.txt')

('shakespeare.txt', <http.client.HTTPMessage at 0x7f2817a3af10>)

In [16]:
path = 'shakespeare.txt'

text = open(path, 'rb').read().decode(encoding='utf-8')

print(text[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [17]:
print('고유한 단어 종류',len(set(text)))

vocab = sorted(set(text))
print(vocab[:10])
print(vocab)

고유한 단어 종류 65
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3']
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [18]:
# word 2 number
char2num = {}
num2char = {}
for ind,char in enumerate(set(text)):
    char2num[char] = ind
    num2char[ind] = char


char2num['C']


62

In [19]:
#text to number
encoded_text = []
for char in text:
    encoded_text.append(char2num[char])

In [21]:
def windowed_dataset(series, window_size, shuffle_buffer, batch_size):
    series = tf.expand_dims(series,-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size +1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda x: x.batch(window_size +1))
    ds = ds.shuffle(shuffle_buffer)
    ds =ds.map(lambda x: (x[:-1], x[1:]))
    return ds.batch(batch_size).prefetch(1)

In [22]:
train_data = windowed_dataset(np.array(encoded_text),100, 10000, 64)

**Model**

In [23]:
vocab_size = len(vocab)
embedding_dim = 256
batch_size = 64
rnn_units = 1024

model = tf.keras.Sequential([
                     tf.keras.layers.Embedding(vocab_size, embedding_dim,batch_input_shape=[batch_size,None]),
                     tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
                     tf.keras.layers.Dense(vocab_size)
                     ])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
lstm_1 (LSTM)                (64, None, 1024)          5246976   
_________________________________________________________________
dense_1 (Dense)              (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [24]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [25]:
model.compile(optimizer='adam', loss=loss, metrics=['acc'])

steps_per_epoch 을 1700
eposch 을 10 이상으로 주어 학습해 보세요! (시간이 많이 소요된다는 점을 참고)

In [28]:
model.fit(train_data, epochs= 1, steps_per_epoch=10)

10/10 [==============================] - 77s 8s/step - loss: 2.3372 - acc: 0.3397


예측을 위한 모델 재정의

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[1, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
])

In [29]:
model.build(tf.TensorShape([1, None]))

In [30]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
lstm_1 (LSTM)                (64, None, 1024)          5246976   
_________________________________________________________________
dense_1 (Dense)              (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [34]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [35]:
model.compile(optimizer='adam', loss=loss, metrics=['acc'])

epoch을 10 이상,
steps_per_epoch 을 1700이상 설정하면 정확도가 높아짐(수행 시간은 길어짐) 

In [37]:
model.fit(train_data, 
          epochs=1, 
          steps_per_epoch=50, 
          )

50/50 [==============================] - 383s 8s/step - loss: 2.3943 - acc: 0.3344
